In [1]:
import numpy as np
#from sklearn.datasets import load_digits
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline


import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()
import tensorflow_hub as hub
import tf_sentencepiece

import nltk
#nltk.download('punkt')

In [ ]:
#teste if gpu is enabled
tf.compat.v1.disable_eager_execution()

with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))
# should be
#   [[22. 28.]
#   [49. 64.]]

In [ ]:
#Universal Sentence encoder (USE)
use_module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/1"

g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed_module = hub.Module(use_module_url)
    embedded_text = embed_module(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

config = tf.ConfigProto(inter_op_parallelism_threads=1,
                   intra_op_parallelism_threads=1)
config.gpu_options.allow_growth = True

session = tf.Session(graph=g, config=config)

session.run(init_op)



def generate_embeddings (messages_in):
    return session.run(embedded_text, feed_dict={text_input: messages_in})

In [3]:
# sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
# plt.scatter(
#     embedding[:, 0],
#     embedding[:, 1],
#     c=[sns.color_palette()[x] for x in penguins.species_short.map({"Adelie":0, "Chinstrap":1, "Gentoo":2})]
# )
# plt.gca().set_aspect('equal', 'datalim')
# plt.title("UMAP o pemguin's beques", fontsize=24)

In [17]:
dataframe.read_csv('')
#dataframe = pd.read_pickle('history_top10_sentence')
#use_embeddings = np.load('embeddings.npy')

#### data cleaning

In [11]:
history = history.drop_duplicates(subset=['title'])

#clear non meaningful words
import re

noiseWords = ['Google Search', '|', '%', '.', ' — ', '/']
big_regex = re.compile('|'.join(map(re.escape, noiseWords)))

dataframe['title'] = dataframe['title'].apply(lambda x : big_regex.sub("", x) )

def remove_non_nouns(lines):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = nltk.word_tokenize(lines)
    return ' '.join( [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] )

history['title'] = history['title'].apply(remove_non_nouns);

#### get most frequent words

In [12]:
#get word frequency
tokens = nltk.tokenize.word_tokenize(' '.join(dataframe["title"]))
freq = nltk.FreqDist(tokens)

In [13]:
freq.most_common(10)

[('Stack', 98),
 ('GitHub', 67),
 ('·', 63),
 ('Learning', 58),
 ('Python', 56),
 ('python', 42),
 ('fastai', 42),
 ('Course', 31),
 ('Forums', 31),
 ('Azure', 30)]

In [ ]:
dataframe['freq_index'] = dataframe['title'].apply(lambda x : max([freq[w] for w in x.split()], default=0) )

In [4]:
#use_embeddings = np.load('embeddings.npy')
compare_to_top = 10
top_n_words_sentence = ' '.join([i[0] for i in freq.most_common(compare_to_top)])
history = history.append({'title': to_10_words_sentence, 'type': 6}, ignore_index=True)

#### nearest neighbour search with annoy

In [35]:
#calculat the distance between USE embeddings
from annoy import AnnoyIndex
nn_tree = AnnoyIndex(512, 'euclidean')
for idx, e in enumerate(use_embeddings):
    nn_tree.add_item(idx, e)
nn_tree.build(10)

True

In [ ]:
idxs = nn_tree.get_nns_by_item(history.last_valid_index(), len(history))

def display_(df):
    pd.set_option("display.max_rows", None)
    from IPython.core.display import display
    display(df)
    
display_(history.reindex(idxs))

In [24]:

colormap = {'0': '#0000ff', '1': '#ffa500', '2': '#34ebcf', '3': '#34eb5f', '4': '#d3eb34', '4': '#d3eb34', '6': '#0d0d0d'}
dataframe['colors'] = [colormap[str(x)] for x in dataframe['type']]

dataframe = dataframe.fillna('')

In [2]:
from bokeh.plotting import figure, curdoc, show
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10
from bokeh.models.widgets.tables import DataTable, TableColumn
from bokeh.layouts import grid
from bokeh.transform import linear_cmap

from bokeh.resources import *
from bokeh.io import output_notebook, reset_output, output_file

output_notebook()

Loading BokehJS ...

In [27]:
def plot_emb_app(doc):
    datasource = ColumnDataSource(dataframe)
    color_mapping = CategoricalColorMapper(factors=[str(t) for t in dataframe.type.unique()], palette=sns.color_palette().as_hex())
    plot_figure = figure(
        title='UMAP',
        plot_width=800,
        plot_height=800,
        tools=['lasso_select','pan','wheel_zoom','box_zoom','reset','save'],
    )

    cicles = plot_figure.circle(
        'index',
        'freq_index',
        source=datasource,
        color='colors',
        #color = linear_cmap(field_name="freq_index", palette=['#d5c7ff', '#3e00fa'], low=min(dataframe['freq_index']), high=max(dataframe['freq_index']) ),
        #color = linear_cmap(field_name="type", palette=['#d5c7ff', '#3e00fa'], low=0, high=4 ),
        line_alpha=0.6,
        fill_alpha=0.6,
        size=4
    )
    
    s2 = ColumnDataSource(data=dict(x=[], y=[], idx=[]))
    
    # SELECT CALLBACK
    def select_callback(attrname, old, new):
        global selected_indxs    
        selected_indxs = new
        #d1 = datasource.data
        #d2 = dict()
        d2 = {col: [] for col in dataframe.columns}
        #species,island,sex,t2,sx,sy = [],[],[],[],[],[]
        
        for i in range(len(selected_indxs)):
            for col in dataframe.columns:
                try:
                    d2[col].append(dataframe[col][selected_indxs[i]])
                except KeyError:
                    pass
        s2.data = d2
    
    datasource.selected.on_change('indices', select_callback)
    
    #table
    columns = []
    for col in dataframe.columns:
        columns.append(TableColumn(field=col, title=col))

    table = DataTable(
        source=s2,
        columns=columns,
        width=800,
        height=400,
        sortable=True,
        selectable=True,
        editable=True,
    )
    doc.add_root(grid([plot_figure, table], nrows=2))

show(plot_emb_app, notebook_url="http://localhost:8888")